
# Modelo de predicción de plan (Smart o Ultra) para usuarios de Megaline.
</div>

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import numpy as np

In [4]:
#Abrimos el fd, observamos que no existan observaciones faltantes y que los datos esten correctos
df = pd.read_csv('users_behavior.csv')
print(df.head())
print()
print(df.info())

   calls  minutes  messages   mb_used  is_ultra
0   40.0   311.90      83.0  19915.42         0
1   85.0   516.75      56.0  22696.96         0
2   77.0   467.66      86.0  21060.45         0
3  106.0   745.53      81.0   8437.39         1
4   66.0   418.74       1.0  14502.75         0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None


In [6]:
#Dividimos el df en variables dependientes e independientes
x= df.drop(['is_ultra'], axis=1)
y=df['is_ultra']
#Segmentamos en entrenamiento, validacion y prueba
x_train, x_temp, y_train, y_temp = train_test_split(x, y, test_size=0.40, random_state=12345)
x_valid, x_test, y_valid, y_test = train_test_split(x_temp, y_temp, test_size = .5, random_state=12345)

In [8]:
#Comenzamos con el modelo de Arboles de clasificación
best_model_1 = None
best_accuracy_1 = 0
best_depth_1 = 0

for depth in range(1, 11):
    model_1 = DecisionTreeClassifier(max_depth= depth, random_state=12345)
    model_1.fit(x_train, y_train)
    predictions_1_valid = model_1.predict(x_valid)
    accuracy_1 = accuracy_score(y_valid, predictions_1_valid)
    if accuracy_1 > best_accuracy_1:
        best_model_1=model_1
        best_accuracy_1=accuracy_1
        best_depth_1=depth
        
print(f"Accuracy del mejor modelo en el conjunto de validación (max_depth = {best_depth_1}): {best_accuracy_1}")        
print()
final_model_1 = DecisionTreeClassifier(max_depth=best_depth_1, random_state=12345)
final_model_1.fit(x_train, y_train)
predictions_1_test = final_model_1.predict(x_test)

final_accuracy_1 = accuracy_score(y_test, predictions_1_test)
print(f'Final accuracy en el conjunto de prueba: {final_accuracy_1:.4f}')

print()

from sklearn.dummy import DummyClassifier

dummy = DummyClassifier(strategy='stratified', random_state=12345)
dummy.fit(x_train, y_train)
dummy_predictions = dummy.predict(x_test)
dummy_accuracy = accuracy_score(y_test, dummy_predictions)

print(f'Accuracy del DummyClassifier (estrategia = "stratified"): {dummy_accuracy:.4f}')




Accuracy del mejor modelo en el conjunto de validación (max_depth = 3): 0.7853810264385692

Final accuracy en el conjunto de prueba: 0.7792

Accuracy del DummyClassifier (estrategia = "stratified"): 0.5365


# Árboles de clasificación

Se hizo el bucle en el conjunto de entrenamiento para escoger el mejor hiperparametro depth, para este caso fué un max_depth=3, en donde se obtuvo un accuracy de .785.

Al aplicar el modelo con max_depth=3 y comprobarlo en el conjunto de prueba, se obtuvo un accuracy de .779, lo que indica que el modelo acertó el plan del cliente el 77.9% de las veces.

Se probó la cordura del modelo comparandola con una prueba dummy donde se haga un modelo sin aprendizaje donde se hacen predicciones al azar pero respetando la proporción original estrategia = "stratified". Dicho modelo tuvo un accuracy de .53, menor al modelo de aprendizaje DecisionTreeClassifier con depth=10; por lo que podemos decir que nuestro modelo final_model_1 no solo adivina; sino que tiene capacidad predictiva real.


In [10]:
#Probamos ahora el modelo Bosque Aleatorio de Clasificación
best_model_2 = None
best_accuracy_2 = 0
best_est_2 = 0
for est in range(1,11):
    model_2=RandomForestClassifier(random_state=12345, n_estimators= est)
    model_2.fit(x_train, y_train)
    predictions_2_valid = model_2.predict(x_valid)
    accuracy_2 = accuracy_score(y_valid, predictions_2_valid)
    if accuracy_2 > best_accuracy_2:
        best_model_2=model_2
        best_accuracy_2=accuracy_2
        best_est_2=est
print(f"Accuracy del mejor modelo en el conjunto de validación (n_estimators = {best_est_2}): {best_accuracy_2}")        
print()
final_model_2 = RandomForestClassifier(random_state=12345, n_estimators= best_est_2)
final_model_2.fit(x_train, y_train)
predictions_2_test = final_model_2.predict(x_test)

final_accuracy_2 = accuracy_score(y_test, predictions_2_test)
print(f'Final accuracy en el conjunto de prueba: {final_accuracy_2:.4f}')



Accuracy del mejor modelo en el conjunto de validación (n_estimators = 10): 0.7853810264385692

Final accuracy en el conjunto de prueba: 0.7807


# Bosque aleatorio de clasificación

Se hizo el bucle en el conjunto de entrenamiento para escoger el mejor hiperparametro n_estimators, para este caso fué un n_estimators=10, en donde se obtuvo un accuracy de .785.

Al aplicar el modelo con n_estimators=10 y comprobarlo en el conjunto de prueba, se obtuvo un accuracy de .78, lo que indica que el modelo acertó el plan del cliente el 78% de las veces.

Se probó la cordura del modelo comparandola con una prueba dummy donde se haga un modelo sin aprendizaje donde se hacen predicciones al azar pero respetando la proporción original estrategia = "stratified". Dicho modelo tuvo un accuracy .53, menor al modelo de aprendizaje DecisionTreeClassifier con depth=10; por lo que podemos decir que nuestro modelo final_model_2 no solo adivina; sino que tiene capacidad predictiva real.

In [12]:
#Probamos ahora el modelo Logistico
best_model_3 = None
best_accuracy_3 = 0
best_C_3 = 0
for c in np.arange(1.0, 11.0):
    model_3=LogisticRegression(random_state=12345, penalty='l2', solver='liblinear', C=c)
    model_3.fit(x_train, y_train)
    predictions_3_valid = model_3.predict(x_valid)
    accuracy_3 = accuracy_score(y_valid, predictions_3_valid)
    if accuracy_3 > best_accuracy_3:
        best_model_3=model_3
        best_accuracy_3=accuracy_3
        best_C_3=c
print(f"Accuracy del mejor modelo en el conjunto de validación (C = {best_C_3}): {best_accuracy_3}")        
print()
final_model_3 = LogisticRegression(random_state=12345, penalty='l2', solver='liblinear', C=best_C_3)
final_model_3.fit(x_train, y_train)
predictions_3_test = final_model_3.predict(x_test)

final_accuracy_3 = accuracy_score(y_test, predictions_3_test)
print(f'Final accuracy en el conjunto de prueba: {final_accuracy_3:.4f}')

Accuracy del mejor modelo en el conjunto de validación (C = 1.0): 0.7589424572317263

Final accuracy en el conjunto de prueba: 0.7403


# Bosque aleatorio de clasificación

Se hizo el bucle en el conjunto de entrenamiento para escoger el mejor hiperparametro C y aplicando la penalización Ridge l2, para este caso fué un C=1.0, en donde se obtuvo un accuracy de .758.

Al aplicar el modelo con C=1.0 y comprobarlo en el conjunto de prueba, se obtuvo un accuracy de .74, lo que indica que el modelo acertó el plan del cliente el 74% de las veces.

Se probó la cordura del modelo comparandola con una prueba dummy donde se haga un modelo sin aprendizaje donde se hacen predicciones al azar pero respetando la proporción original estrategia = "stratified". Dicho modelo tuvo un accuracy de .53 menor al modelo de aprendizaje DecisionTreeClassifier con depth=10; por lo que podemos decir que nuestro modelo final_model_3 no solo adivina; sino que tiene capacidad predictiva real, aunque no llega a la exactitud deseada.

In [14]:
print(f'Final Accuracy del modelo Árboles de clasificación: {final_accuracy_1:.4f}')
print()
print(f'Final Accuracy del modelo Bosque Aleatorio de Clasificación: {final_accuracy_2:.4f}')
print()
print(f'Final Accuracy del modelo Logístico: {final_accuracy_3:.4f}')

Final Accuracy del modelo Árboles de clasificación: 0.7792

Final Accuracy del modelo Bosque Aleatorio de Clasificación: 0.7807

Final Accuracy del modelo Logístico: 0.7403


# Conclusiónes
Posterior a hacer pruebas con 3 modelos con distintos hiperparámetros, todos los modelos aprobaron la cordura del modelo comparandola con una prueba dummy


Se obtuvo que tanto los modelos Árboles de clasificación y Bosque Aleatorio de Clasificación obtuvieron un umbral de exactitud mayor al objetivo ">75", mientras que el Accuracy del modelo Logístico se encuentra a niveles cercanos y podría ser aceptable en varios casos, en este al no alcanzar el objetivo se prefieren los otros 2.


El Accuracy del modelo Árboles de clasificación: 0.7792 es menor al Accuracy del modelo Bosque Aleatorio de Clasificación: 0.7807; por lo que en este modelo con estas variables se preferiría el uso del modelo Bosque Aleatorio de Clasificación.

